In [1]:
import gzip
import ast
import json
import pandas as pd
from datetime import datetime

In [2]:
def crearDFJSON(ruta):
    data = []
    with gzip.open(ruta, 'rb') as f:
        for line in f:
            data.append(json.loads(line))
    f.close
    return pd.DataFrame(data)

def crearDFAST(ruta):
    data = []
    with gzip.open(ruta, 'rb') as f:
        for line in f:
            data.append(ast.literal_eval(line.decode('utf-8')))
    f.close
    return pd.DataFrame(data)

def desanidarDF(df, columna):
    df_desanidado = df.explode(columna)
    return df_desanidado

def desanidar(df, columna):
    dfDesanidado = pd.json_normalize(df[columna])
    df = df.reset_index(drop=True)
    df = pd.concat([df, dfDesanidado], axis=1)
    df = df.drop(columna, axis=1)
    return df

In [3]:
steamGamesDF = crearDFJSON('../data/steam_games.json.gz')
usersItems = crearDFAST('../data/users_items.json.gz')

In [4]:
#eliminar nulos e ajustar el indice
steamGamesDF = steamGamesDF.dropna().reset_index(drop=True)
playTimeGenreDF = steamGamesDF[['genres', 'release_date', 'id']]
playTimeGenreDF = desanidarDF(columna='genres', df=playTimeGenreDF)

In [5]:
playTimeGenreDF.columns = ['Generos', 'Fecha_Lanzamiento', 'IdItem']

In [6]:
playTimeGenreDF

,Generos,Fecha_Lanzamiento,IdItem
0,Action,2018-01-04,761140
0,Casual,2018-01-04,761140
0,Indie,2018-01-04,761140
0,Simulation,2018-01-04,761140
0,Strategy,2018-01-04,761140
...,...,...,...
22528,Indie,2018-01-04,610660
22528,Racing,2018-01-04,610660
22528,Simulation,2018-01-04,610660
22529,Casual,2017-09-02,658870


In [7]:
columnas_elimnar = ['user_id','steam_id', 'user_url', 'items_count']
usersItems = usersItems.drop(columnas_elimnar, axis=1).reset_index(drop=True)
usersItems = desanidarDF(usersItems, 'items')
usersItems = desanidar(usersItems, 'items')
columnas_elimnar = ['item_name','playtime_2weeks']
usersItems = usersItems.drop(columnas_elimnar, axis=1).reset_index(drop=True)
usersItems.columns = ['IdItem', 'Tiempo_Jugado']
df_agrupado = usersItems.groupby('IdItem')['Tiempo_Jugado'].sum().reset_index()

In [8]:
df_agrupado

,IdItem,Tiempo_Jugado
0,10,17386015.0
1,100,311999.0
2,10000,63046.0
3,1002,894.0
4,10040,0.0
...,...,...
10973,99890,128136.0
10974,9990,2110.0
10975,99900,17259605.0
10976,99910,426680.0


In [9]:
# Unir los DataFrames por la columna 'IdItem'
playTimeGenreDF = pd.merge(playTimeGenreDF, df_agrupado, on='IdItem', how='inner')

In [10]:
columnas_elimnar = ['IdItem']
playTimeGenreDF = playTimeGenreDF.drop(columnas_elimnar, axis=1).reset_index(drop=True)

In [11]:
playTimeGenreDF

,Generos,Fecha_Lanzamiento,Tiempo_Jugado
0,Action,1997-06-30,9319.0
1,Indie,1997-06-30,9319.0
2,Racing,1997-06-30,9319.0
3,Action,1998-11-08,2682852.0
4,Strategy,2006-07-06,27397.0
...,...,...,...
20572,Strategy,2003-07-01,3393.0
20573,RPG,2003-07-01,3393.0
20574,Indie,2003-07-01,3393.0
20575,Action,2004-03-16,349119.0


In [12]:
def PlayTimeGenre(genero: str, df):
    # Filtrar el DataFrame para obtener solo las filas que corresponden al género proporcionado
    filtered_df = df[df['Generos'] == genero].copy()  # Crear una copia explícita
    
    # Extraer el año de la columna 'Fecha_Lanzamiento' (sin importar el formato)
    filtered_df['Año_Lanzamiento'] = pd.to_datetime(filtered_df['Fecha_Lanzamiento'], errors='coerce').dt.year
    
    # Eliminar filas con años no válidos
    filtered_df = filtered_df.dropna(subset=['Año_Lanzamiento'])
    
    # Agrupar por año y sumar las horas jugadas para cada año
    horas_jugadas_por_año = filtered_df.groupby('Año_Lanzamiento')['Tiempo_Jugado'].sum()
    
    if horas_jugadas_por_año.empty:
        return None  # Retorna None si no hay datos para el género dado
    
    # Encontrar el año con la suma más alta de horas jugadas
    año_con_mas_horas = horas_jugadas_por_año.idxmax()

    return año_con_mas_horas

In [13]:
genero = 'Action'  # El género deseado
año = PlayTimeGenre(genero, playTimeGenreDF)
print(f"Año con más horas jugadas para el género {genero}: {año}")

Año con más horas jugadas para el género Action: 2012.0


In [14]:
playTimeGenreDF.to_parquet('PlayTimeGenreParquet.parquet', index=False)